In [1]:
import pyaniasetools as pya
from rdkit import Chem
from rdkit.Chem import AllChem
import hdnntools as hdt
import numpy as np

van Der Waals correction will be unavailable. Please install ased3


In [2]:
netdir = '/scratch/Gits/ANI-Networks/networks/al_networks/ANI-AL-0808.0303.0400/'
cns = netdir+'train0/rHCNOSFCl-4.6A_16-3.1A_a4-8.params'
sae = netdir+'train0/sae_wb97x-631gd.dat'
nnf = netdir+'train'
Nn = 5

In [3]:
cmp = pya.anienscomputetool(cns,sae,nnf,Nn,0)

In [4]:
m = Chem.MolFromSmiles('OC(=O)COC(=O)CC1=CC=CC=C1NC1=C(Cl)C=CC=C1Cl')
m2=Chem.AddHs(m)
# run distance geometry 10 times
cids = AllChem.EmbedMultipleConfs(m2, useExpTorsionAnglePrefs=True, useBasicKnowledge=True, numConfs=50)
print(len(cids))
for cid in cids:
    print('Confid:',cid)
    _ = cmp.optimize_rdkit_molecule(m2, cid=cid)
rmslist = []
AllChem.AlignMolConformers(m2, RMSlist=rmslist)
print(rmslist)

50
Confid: 0
Confid: 1
Confid: 2
Confid: 3
Confid: 4
Confid: 5
Confid: 6
Confid: 7
Confid: 8
Confid: 9
Confid: 10
Confid: 11
Confid: 12
Confid: 13
Confid: 14
Confid: 15
Confid: 16
Confid: 17
Confid: 18
Confid: 19
Confid: 20
Confid: 21
Confid: 22
Confid: 23
Confid: 24
Confid: 25
Confid: 26
Confid: 27
Confid: 28
Confid: 29
Confid: 30
Confid: 31
Confid: 32
Confid: 33
Confid: 34
Confid: 35
Confid: 36
Confid: 37
Confid: 38
Confid: 39
Confid: 40
Confid: 41
Confid: 42
Confid: 43
Confid: 44
Confid: 45
Confid: 46
Confid: 47
Confid: 48
Confid: 49
[2.2561698466953484, 1.830117374070418, 2.000370546810922, 2.419403375874553, 2.44550785465001, 2.1373976176847207, 2.3231307085116617, 0.8606803991540505, 1.6615644488037156, 1.9547217913284627, 1.4147772824731413, 1.9719629307309046, 2.652593264695597, 2.4868333791582273, 1.5952031021245472, 1.5175666090363984, 1.0358469680190825, 1.871934098219737, 2.747646649948246, 1.1617214151412192, 0.4177668328451919, 2.3230716046007887, 1.3916569774672891, 2.31

In [5]:
E,V = cmp.energy_rdkit_conformers(m2,cids)
E = hdt.hatokcal*(E-E.min())
efile = open('/nh/nest/u/jsmith/scratch/Research/confs_test/fentanyl.xyz')
idx = np.argsort(E)
print(E,idx)

[3.28542581e+00 5.80821927e+00 8.79861339e+00 4.35814106e+00
 4.33479864e+00 1.12659352e+00 6.64961562e+00 8.18943641e-01
 3.45528823e+00 3.45427790e+00 1.74996755e+00 8.01240300e+00
 7.63224186e+00 2.84882246e+00 1.05043338e+01 3.28627204e+00
 1.25357129e+01 7.63223999e+00 3.28542908e+00 2.39103108e+00
 4.33473599e+00 3.28547817e+00 8.18917459e-01 9.71090661e+00
 9.71240551e+00 7.23372126e+00 9.99849427e+00 1.88011649e+00
 3.28552679e+00 1.42883870e+01 1.19685848e+01 8.01254139e+00
 3.08539501e+00 1.87958023e+00 0.00000000e+00 8.18957667e-01
 8.69356619e+00 3.28592606e+00 3.08549226e+00 3.45493431e+00
 4.16102293e-05 3.88040200e+00 1.03282032e+01 9.54450251e+00
 9.98678870e+00 1.12671508e+00 1.66441060e-04 8.68613058e+00
 8.19147952e-01 6.12633250e+00] [34 40 46 22  7 35 48  5 45 10 33 27 19 13 32 38  0 18 21 28 37 15  9 39
  8 41 20  4  3  1 49  6 25 17 12 11 31 47 36  2 43 23 24 44 26 42 14 30
 16 29]


In [6]:
X,S = pya.__convert_rdkitconfs_to_nparr__(m2)
Xs = X[idx]
Es = E[idx]
Vs = V[idx]

In [8]:
comments = [str(es)+','+str(vs) for es,vs in zip(Es,Vs)]
hdt.writexyzfilewc('/nh/nest/u/jsmith/scratch/Research/confs_test/aceclofenac.xyz',Xs,S,comments)